In [12]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from tqdm import tqdm
from langchain_openai import AzureOpenAIEmbeddings
import json
import numpy as np
import torch
import time
import random
import ast
import os

Let's fetch the dataframes.

In [7]:
from data_extractor import Extractor
extractor_instance = Extractor()
extractor_instance.extract(location="./data/results/df_combined.csv")

df_combined = pd.read_csv("data/results/df_combined.csv")
df_combined.sample(n=3)

Comp domain df length: 1912  - VTT domain df length: 4927
Example of text to compare: FiR 1 - FiR 1 is a Triga-type nuclear research reactor located in Otaniemi, Espoo, Finland, decommissioned by VTT after serving since 1962, used for nuclear research, training, and medical radiation therapy. | Developed by VTT Technical Research Centre of Finland Ltd.


,Unnamed: 0.1,Document number,VAT id,relationship description,source id,source type,source english_id,source description,relationship type,target id,target type,target english_id,target description,Link Source Text,Source Text,text_to_compare,Unnamed: 0,Source Company
3634,3634,VTT901,FI02134547,"Fortum Power and Heat Oy, via its Bio2X bioref...",Handprint Guideline,Innovation,Handprint Guideline,An updated scientific guideline created by VTT...,DEVELOPED_BY,FI14636114,Organization,Fortum Power and Heat Oy,An industrial Finnish partner participating in...,https://www.vttresearch.com/en/news-and-ideas/...,Skip to main content Beyond the obvious Open m...,Handprint Guideline - An updated scientific gu...,NaN,NaN
2098,2098,VTT531,FI26277526,Metsä Spring is involved as an industrial part...,ValCel - Value for Cellulosics,Innovation,ValCel - Value for Cellulosics,An innovation project developing new sustainab...,DEVELOPED_BY,temp_53,Organization,Metsä Spring,An industrial partner contributing to the ValC...,https://www.vttresearch.com/fi/project_news/va...,Hyppää pääsisältöön Beyond the obvious ...,ValCel - Value for Cellulosics - An innovation...,NaN,NaN
10,10,VTT6,FI16801400,Destaclean contributed as a partner to the dev...,logistics of circular economy,Innovation,logistics of circular economy,A new process innovation that addresses the ch...,DEVELOPED_BY,FI14810370,Organization,Destaclean,A company participating in the AARRE project f...,https://www.vttresearch.com/en/news-and-ideas/...,Skip to main content Beyond the obvious Open m...,logistics of circular economy - A new process ...,NaN,NaN


In [ ]:
from embedding_generator import EmbeddingGenerator, LocalEmbeddingProvider, OpenAIEmbeddingProvider

# Configuration
METHOD = "openai"  # Supports local and openai
MODEL_NAME = "all-mpnet-base-v2"  # For local embeddings
MODEL_KEY = "gpt-4.1-mini"  # For OpenAI embeddings, I think only this works

# File paths
input_file = "data/results/df_combined.csv"
output_file = "data/results/embeddings.csv"

provider = OpenAIEmbeddingProvider(MODEL_KEY) if METHOD == "openai" else LocalEmbeddingProvider(MODEL_NAME)
generator = EmbeddingGenerator(provider)

result_df = None
if os.path.exists(output_file):
    result_df = pd.read_csv(output_file)
    print(f"Loaded {len(result_df)} pairs from {output_file}")
else:
    result_df = generator.generate_embeddings(input_file, output_file)

result_df["embedding"] = result_df["embedding"].apply(ast.literal_eval)

2025-05-31 15:13:18,548 - INFO - Initialized OpenAI embedding model with key: gpt-4.1-mini


Loaded 6839 pairs from data/results/embeddings.csv


In [9]:
result_df.head()

,Unnamed: 0.1,Document number,VAT id,relationship description,source id,source type,source english_id,source description,relationship type,target id,...,target english_id,target description,Link Source Text,Source Text,text_to_compare,Unnamed: 0,Source Company,embedding,embedding_provider,embedding_dimension
0,0,VTT0,FI10292588,"FiR 1 nuclear research reactor was developed, ...",FiR 1,Innovation,FiR 1,FiR 1 is a Triga-type nuclear research reactor...,DEVELOPED_BY,FI26473754,...,VTT Technical Research Centre of Finland Ltd.,VTT is a Finnish research and innovation partn...,https://www.vttresearch.com/en/news-and-ideas/...,Skip to main content Beyond the obvious Open m...,FiR 1 - FiR 1 is a Triga-type nuclear research...,NaN,NaN,"[-0.018910497426986694, 0.017533520236611366, ...",openai_gpt-4.1-mini,3072
1,1,VTT0,FI10292588,Centre for Nuclear Safety is being developed a...,Centre for Nuclear Safety,Innovation,Centre for Nuclear Safety,A modern research facility under construction ...,DEVELOPED_BY,FI26473754,...,VTT Technical Research Centre of Finland Ltd.,VTT is a Finnish research and innovation partn...,https://www.vttresearch.com/en/news-and-ideas/...,Skip to main content Beyond the obvious Open m...,Centre for Nuclear Safety - A modern research ...,NaN,NaN,"[-0.009751166217029095, 0.006150579079985619, ...",openai_gpt-4.1-mini,3072
2,2,VTT3,FI08932048,The innovation approach 'Beyond the obvious' i...,Beyond the obvious,Innovation,Beyond the obvious,An innovation approach promising to provide so...,DEVELOPED_BY,FI26473754,...,VTT Technical Research Centre of Finland Ltd,A visionary research and innovation partner fo...,https://www.vttresearch.com/en/news-and-ideas/...,Skip to main content Beyond the obvious Open m...,Beyond the obvious - An innovation approach pr...,NaN,NaN,"[-0.02210899069905281, -0.002120873425155878, ...",openai_gpt-4.1-mini,3072
3,3,VTT4,FI01111693,Data-Driven Bioeconomy project is developed by...,Data-Driven Bioeconomy project,Innovation,Data-Driven Bioeconomy project,An innovation using Big Data for sustainable u...,DEVELOPED_BY,FI26473754,...,VTT Technical Research Centre of Finland,A Finnish research and innovation partner work...,https://www.vttresearch.com/en/news-and-ideas/...,Skip to main content Beyond the obvious Open m...,Data-Driven Bioeconomy project - An innovation...,NaN,NaN,"[-0.015033945441246033, 0.031899139285087585, ...",openai_gpt-4.1-mini,3072
4,4,VTT4,FI01111693,Data-Driven Bioeconomy project's forestry pilo...,Data-Driven Bioeconomy project,Innovation,Data-Driven Bioeconomy project,An innovation using Big Data for sustainable u...,DEVELOPED_BY,temp_1141,...,MHG Systems,An organization leading pilots developing fore...,https://www.vttresearch.com/en/news-and-ideas/...,Skip to main content Beyond the obvious Open m...,Data-Driven Bioeconomy project - An innovation...,NaN,NaN,"[-0.014490003697574139, 0.016991155222058296, ...",openai_gpt-4.1-mini,3072


In [10]:
threshold = 0.70

similar_pairs = []

#result_df = result_df.drop_duplicates(subset=["source description"]).reset_index(drop=True)

combined_embeddings = result_df["embedding"].tolist()
combined_texts = result_df["text_to_compare"].tolist()

start_time = time.time()

# Convert embeddings to a tensor
embeddings_tensor = torch.stack([torch.tensor(e) for e in combined_embeddings])

# Compute cosine similarity matrix
cos_sim_matrix = util.cos_sim(embeddings_tensor, embeddings_tensor)

# Get indices of upper triangle (i < j)
triu_indices = torch.triu_indices(cos_sim_matrix.size(0), cos_sim_matrix.size(1), offset=1)

# Get similarity scores for upper triangle
sim_scores = cos_sim_matrix[triu_indices[0], triu_indices[1]]

# Filter by threshold
mask = sim_scores > threshold
i_indices = triu_indices[0][mask].tolist()
j_indices = triu_indices[1][mask].tolist()
scores = sim_scores[mask].tolist()

# Collect similar pairs
similar_pairs = [(i, j, score) for i, j, score in zip(i_indices, j_indices, scores)]

end = time.time()
print(f"Similarity computation took {end - start_time:.2f} seconds")
num_samples = 5
sampled_pairs = random.sample(similar_pairs, min(num_samples, len(similar_pairs)))

for idx, (i, j, score) in enumerate(sampled_pairs[:5]):  # Print first 5 examples
    print(f"Pair {idx+1}:")
    print(f"  Index {i} → {combined_texts[i]}")
    print(f"  Index {j} → {combined_texts[j]}")
    print(f"elem : {result_df.iloc[i]}")
    print(f"  Similarity: {score:.4f}\n")

TypeError: new(): invalid data type 'str'